In [1]:
import pandas as pd
import numpy as np

In [37]:
data_close = pd.read_csv('data/Close.csv').T[1:]
data_close.index = data_close.index.astype('datetime64').rename('date')
data_close = data_close.astype('float64')

data_open = pd.read_csv('data/Open.csv').T[1:]
data_open.index = data_open.index.astype('datetime64').rename('date')
data_open = data_open.astype('float64')

data_low = pd.read_csv('data/Low.csv').T[1:]
data_low.index = data_low.index.astype('datetime64').rename('date')
data_low = data_low.astype('float64')

data_high = pd.read_csv('data/High.csv').T[1:]
data_high.index = data_high.index.astype('datetime64').rename('date')
data_high = data_high.astype('float64')


In [7]:
def get_returns(data):
    return (data / data.shift(1))

In [8]:
def correct_wrong_values(data, scale):
    temp = np.array(get_returns(data)).T
    err_coord_low = np.where(temp <= (1 / scale))
    err_coord_high = np.where(temp >= scale)
    for i in range(len(err_coord_low[0])):
        id_low_0 = err_coord_low[0][i]
        id_low_1 = err_coord_low[1][i]
        
        id_high_0 = err_coord_high[0][i]
        id_high_1 = err_coord_high[1][i]

        if 0.5 < data[id_high_0][id_low_1] / data[id_high_0][id_high_1 + 1] < 2:        # if the price has dropped
            data[id_high_0][id_high_1] = data[id_high_0][id_high_1 + 1]

        if 0.5 < data[id_high_0][id_high_1] / data[id_high_0][id_low_1 + 1] < 2:        # If the price went up
            data[id_high_0][id_low_1] = data[id_high_0][id_low_1 + 1]
    return pd.DataFrame(data)

In [33]:
def get_correct_value(data, newname):
    scale = 9
    print(np.where(get_returns(data) >= scale)[0])
    print(np.where(get_returns(data) >= scale)[1])
    print('correct wrong values')
    data = correct_wrong_values(data, scale)
    print(np.where(get_returns(data) >= scale)[0])
    print(np.where(get_returns(data) >= scale)[1])
    data.to_csv('data_correct/' + newname + '_correct.csv')

In [34]:
get_correct_value(data_close, 'Close')

[ 155  178  256  274  306  567  680  748  856  902  904  983 1047]
[1528  592  569  772 1657 1315  185  592 1805  178 2313 2144  565]
correct wrong values
[256 274]
[569 772]


In [35]:
get_correct_value(data_open, 'Open')

[ 227  256  274  364  879  887 1019]
[544 569 772 592 842 185 178]
correct wrong values
[256 274]
[569 772]


In [38]:
get_correct_value(data_low, 'Low')

[  17   51  146  156  156  158  174  227  256  274  476  484  588  776
  888  894 1063]
[ 921  544 1844  129  544 2232  695  544  569  772 2144 2144 1315 2144
  185  185  611]
correct wrong values
[158 256 274]
[2232  569  772]


In [42]:
get_correct_value(data_high, 'High')

[101 256 274 431 589 591 663 725 754 867 886 959]
[ 544  569  772 2158 1315 1157  178  592  178 1805  185 1178]
correct wrong values
[256 274 591]
[ 569  772 1157]
